In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [3]:
class RBMs():
    
    """
    visible_unit      := v_i
    visible_vector    := v_v
    visible_bias      := a_i
    hidden_unit       := h_j
    hidden_vector     := h_v
    hidden_bias       := b_j
    weight_matrix     := w_ij
    learning_rate     := l_r
    
    
    
    """
    def __init__ (self, v_i, a_b, v_v, h_j, b_j, h_v,  w_ij, l_r):
        
        self.v_i   = v_i
        self.v_b   = a_b
        self.v_v   = v_v
        self.h_j   = h_j
        self.h_b   = b_j
        self.h_v   = h_v
        self.w_ij  = w_ij
        self.l_r   = 0.001
        
    # Energy of the RBM between the visible and hidden units that gives us a joint configuration   
    def Energy_fuction(self, v_i, a_i, h_j, b_j, w_ij):
        
        Energy = - sum(a_i , v_i) - sum(b_j , h_j) 
        - sum(v_i , h_j , w_ij)
            
        return Energy
    
    #Probability between the visible and hidden units
    def Probability(self, Energy, Zet):
        
        probability = 1/Zet * (np.exp(-Energy))
        
        return probability
    
    # The partition function between the visible and hidden units, gives all possible connection
    def Partition_function(self, Energy):
        
        Zet = sum(np.exp(-Energy))
        
        return Zet
    
    # Activation fuction    
    def sigmoid(self, x):
        
        sigma = 1/(1+ np.exp(-x))
        
        return sigma
    
    
    # Forward pass
    # Given a randomly selected visible_vector(v_v),  we set the hidden unit to 1 with probability;
    def Unbiased_sample_hidden_unit(self, b_j, v_i, w_ij):
        
        prob_hidden = sigma(b_j + sum(v_i * w_ij))
 
        return prob_hidden
    
    # Backward pass
    #Given hidden_vector, we set the visible_unit to 1 with probability;
    def Unbiased_sample_visible_unit(self, a_i, h_j, w_ij):
        
        prob_visible = sigma(a_i + sum(h_j * w_ij))
        
        return prob_visible
        
    
    
    # Constractive Divergence; n=1
    
    
    

        

In [4]:
class RBM():
    def __init__(self, input_size, output_size, learning_rate, epochs, batchsize):
        
        self._input_size = input_size
        self._output_size = output_size
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.batchsize = batchsize
        
        
        self.w = np.zeros([input_size, output_size])
        self.hb = np.zeros([output_size])
        self.vb = np.zeros([input_size])
        
    
    
    # Activation fuction    
    def sigmoid(self, x):
        sigma = 1/(1+ np.exp(-x))
        return sigma
    
    #Forward pass    
    def prob_h_given_v(self, visible, w, hb):
        a = np.dot(visible, w) + hb
        print(a)
        return self.sigmoid(a)
    
    #Backward pass
    def prob_v_given_h(self, hidden, w, vb):
        
        return self.sigmoid(np.dot(hidden, np.transpose(w)) + vb)
    
    #Sampling function
    def sample_prob(self, probs):
        
        return relu(np.sign(probs - np.random.uniform(np.shape(probs))))
    
    
    

In [5]:
rbm = RBM(5, 3, 0.01, 10, 100)

In [6]:
rbm

In [7]:
RBM.sigmoid(2, 2)

0.8807970779778823

In [8]:
rbm.prob_h_given_v(2,3,4)

10


0.9999546021312976

In [47]:
class RBM():
    def __init__(self, num_visible, num_hidden):
        
        self.num_visible = num_visible
        self.num_hidden = num_hidden
        
        #Initialize a weight matrix, of dimension (num_visible * num_hidden), using
        # a uniform distribution -sqrt(6./(num_hidden + num_visible))
        # and sqrt(6./(num_hidden + num_visible)). One could vary the standard deviation by
        # multiplying the interval with appropriate value.
        # Here we Initialize the weights with mean 0 and standard deviation 0.1
        
        np_rng = np.random.RandomState(1234)
        
        self.weights = np.asarray(np_rng.uniform(low=-0.1*np.sqrt(6./(num_hidden+num_visible)),
                                                high = 0.1*np.sqrt(6./(num_hidden+ num_visible)),
                                                size = (num_visible, num_hidden)))
        
        #Insert weights for the bias units into the first row and first column.
        self.weights = np.insert(self.weights, 0, 0, axis=0)
        self.weights = np.insert(self.weights, 0, 0, axis=1)
        
    def _logistic(self, x):
                
        return 1.0/(1+ np.exp(-x))
            
          
        
        
    def train(self, data, max_epochs=1000, learning_rate=0.1):
        num_examples = data.shape[0]
        #Insert bias units of 1 into the first column.
        data = np.insert(data, 0, 1, axis=1)
        for epoch in range(max_epochs):
            # Clamp to the data and sample from the hidden units
            # (This is the 'positive CD phase', aka the reality phase)
            
            pos_hidden_activations = np.dot(data, self.weights)
            pos_hidden_probs = self._logistic(pos_hidden_activations)
            pos_hidden_probs[:,0] = 1 # fix the bias unit
            pos_hidden_state = pos_hidden_probs > np.random.rand(num_examples, self.num_hidden +1)
            
            # NB: we are using the activation *probabilities* of the hidden states,
            # not the hidden states themselves, when computing associations
            # We  could use the states
            
            pos_associations = np.dot(data.T, pos_hidden_probs)
            
            # Resconstruct the visible units and sample again from the hidden units
            # This is the 'negative CD phase', aka day-dreaming phase
            
            neg_visible_activations = np.dot(pos_hidden_state, self.weights.T)
            neg_visible_probs = self._logistic(neg_visible_activations)
            neg_visible_probs[:,0] = 1 # fix the buas unit
            neg_hidden_activations = np.dot(neg_visible_probs, self.weights)
            neg_hidden_probs = self._logistic(neg_hidden_activations)
            
            # NB: again we're using the activation *probabilities* when
            # computing associations, not the states themselves
            
            neg_associations = np.dot(neg_visible_probs.T, neg_hidden_probs)
            
            #Update weights
            
            self.weights += learning_rate * ((pos_associations - neg_associations)/ num_examples)
            
            error = np.sum((data - neg_visible_probs)**2)
            
            print("Epoch %s: error is %s" %(epoch, error))
            
    def run_visible(self, data):
            
        num_examples = data.shape[0]
            
        # Create a matrix, where each row is to be the hidden units(plus a bias unit),
        # sampled from a training examples
            
        hidden_states = np.ones((num_examples, self.num_hidden + 1))
            
        # Insert bias units of 1 into the first column of data
        data = np.insert(data, 0, 1, axis=1)
            
        #Calculate the activation of the hidden units
        hidden_activations = np.dot(data, self.weights)
            
        #Calculate the probabilities of turning the hidden units on
        hidden_probs = self._logistic(hidden_activations)
            
        #Turn the hidden units on with their specified probabilities
        hidden_states[:,:] = hidden_probs > np.random.rand(num_examples, self.num_hidden + 1)
            
        #Always fix the bias unit to 1
        hidden_states[:,0] =1
            
        #Ignore the bias units
        hidden_states = hidden_states[:,1:]
            
        return hidden_states
        
    def run_hidden(self, data):
            
        num_examples = data.shape[0]
            
        # Create a matrix, where each row is to be the visible units(plus a bias unit)
        # Sampled from a training example
            
        visible_states = np.ones((num_examples, self.num_visible +1))
            
        # Insert bias units of 1 into the first column of data
        data = np.insert(data, 0, 1, axis=1)
            
        # Calculate the activations of the visible units
        visible_activations = np.dot(data, self.weights.T)
            
        # Calculate the probabilities of turning the visible units on
        visible_probs = self._logistic(visible_activations)
            
        # Turn the visible units on with their specified probabilities
        visible_states[:,:] = visible_probs > np.random.rand(num_examples, self.num_visible + 1)
            
        # Always fix the bias unit to 1
        visible_states[:,0] = 1
            
        # Ignore the bias units
        visible_states = visible_states[:,1:]
            
        return visible_states
        
    def daydream(self, num_samples):
            
        # Create a matrix, where each row is to be a sample of the visible
        # units (with an extra bias unit), initialized to all ones
            
        samples = np.ones((num_samples, self.num_visible + 1))
            
        # Take the first sample from a uniform distribution.
        samples[0,1:] = np.random.rand(self.num_visible)
            
        # Start the alternating Gibbs sampling
        # NB: we keep the hidden units binary states, but leave the visible units
        # as real probabilities
            
        for i in range(1, num_samples):
            visible = samples[i-1,:]
                
            # Calculate the activation of the hidden units.
            hidden_activation = np.dot(visible, self.weights)
                
            # Calculate the probabilities of turning the hidden units on
            hidden_probs = self._logistic(hidden_activations)
                
            # Turn the hidden units on with their specified probabilities
            hidden_states = hidden_probs > np.random.rand(self.num_hidden + 1)
                
            # Always fix the bias unit to 1
            hidden_states[0] = 1
                
            # Recalculate the probabilities that the visible units are on
            visible_activations = np.dot(hidden_states, self.weights.T)
            visible_probs = self._logistic(visible_activations)
            visible_states = visible_probs > np.random.rand(self.num_visible + 1)
            samples[i,:] = visible_states
                
            return samples[:,1:]
            
            
           
            
        
            

In [48]:
r = RBM(num_visible = 6, num_hidden = 2)

In [49]:
training_data = np.array([[1,1,1,0,0,0],[1,0,1,0,0,0],[1,1,1,0,0,0],[0,0,1,1,1,0], [0,0,1,1,0,0],[0,0,1,1,1,0]])

In [50]:
r.train(training_data, max_epochs = 5000)

Epoch 0: error is 9.029663077646742
Epoch 1: error is 8.890439145990268
Epoch 2: error is 8.64927950825834
Epoch 3: error is 8.418088284334743
Epoch 4: error is 8.181756781978509
Epoch 5: error is 8.060080666393594
Epoch 6: error is 8.05427170135888
Epoch 7: error is 7.803141865515834
Epoch 8: error is 7.61076348407353
Epoch 9: error is 7.5128809429335845
Epoch 10: error is 7.255660335219702
Epoch 11: error is 7.109351115838238
Epoch 12: error is 7.141605069750923
Epoch 13: error is 7.085624609201376
Epoch 14: error is 6.829666358162809
Epoch 15: error is 6.7595148092469675
Epoch 16: error is 6.670711853232574
Epoch 17: error is 7.09869847266658
Epoch 18: error is 6.676109177111667
Epoch 19: error is 6.828416195668298
Epoch 20: error is 6.492269928786855
Epoch 21: error is 6.613475855320648
Epoch 22: error is 6.500568669457071
Epoch 23: error is 6.61713016687218
Epoch 24: error is 6.349067106445133
Epoch 25: error is 6.383206834148204
Epoch 26: error is 6.417935301595518
Epoch 27: erro

Epoch 1602: error is 1.345079712285369
Epoch 1603: error is 0.9485182100212934
Epoch 1604: error is 0.9435938011339222
Epoch 1605: error is 1.3469933415151514
Epoch 1606: error is 0.9399703578380455
Epoch 1607: error is 1.347769331139124
Epoch 1608: error is 1.3473445884239377
Epoch 1609: error is 1.34693630922029
Epoch 1610: error is 1.3465439564379507
Epoch 1611: error is 0.9399137530552689
Epoch 1612: error is 0.9352142144899931
Epoch 1613: error is 0.9306624692156822
Epoch 1614: error is 1.3497574207684075
Epoch 1615: error is 1.3492594228065862
Epoch 1616: error is 1.3487802275491048
Epoch 1617: error is 1.3483192466668188
Epoch 1618: error is 1.347875904367823
Epoch 1619: error is 1.3474496375567977
Epoch 1620: error is 1.3470398959562795
Epoch 1621: error is 1.346646142192302
Epoch 1622: error is 1.5449988639295509
Epoch 1623: error is 0.9407144186522338
Epoch 1624: error is 1.3464914785834095
Epoch 1625: error is 1.3461155455006566
Epoch 1626: error is 0.9381460829960423
Epoch 

Epoch 2764: error is 0.6949547639386842
Epoch 2765: error is 0.6948364132680706
Epoch 2766: error is 0.6947196479667469
Epoch 2767: error is 0.6946044389657815
Epoch 2768: error is 0.6944907578329351
Epoch 2769: error is 0.6943785767562755
Epoch 2770: error is 0.6942678685282794
Epoch 2771: error is 0.6941586065304024
Epoch 2772: error is 0.694050764718105
Epoch 2773: error is 0.6939443176063167
Epoch 2774: error is 0.6938392402553257
Epoch 2775: error is 0.6937355082570786
Epoch 2776: error is 0.6936330977218776
Epoch 2777: error is 0.693531985265462
Epoch 2778: error is 0.6934321479964612
Epoch 2779: error is 0.6933335635042079
Epoch 2780: error is 0.6932362098468979
Epoch 2781: error is 0.693140065540089
Epoch 2782: error is 1.5289455981762328
Epoch 2783: error is 0.6935462003673449
Epoch 2784: error is 0.6934443543867804
Epoch 2785: error is 0.6933437992223291
Epoch 2786: error is 0.6932445121364167
Epoch 2787: error is 1.527898639904957
Epoch 2788: error is 0.6936603945290009
Epoc

Epoch 4119: error is 0.6744811856025483
Epoch 4120: error is 0.6744740477182511
Epoch 4121: error is 0.6744669327570353
Epoch 4122: error is 0.6744598405731896
Epoch 4123: error is 0.674452771022125
Epoch 4124: error is 0.6744457239603499
Epoch 4125: error is 0.6744386992454491
Epoch 4126: error is 0.6744316967360615
Epoch 4127: error is 0.6744247162918617
Epoch 4128: error is 1.6122058332829845
Epoch 4129: error is 0.6743485825896619
Epoch 4130: error is 0.6743421748445062
Epoch 4131: error is 0.6743357842663138
Epoch 4132: error is 0.674329410755745
Epoch 4133: error is 0.6743230542140222
Epoch 4134: error is 0.6743167145429269
Epoch 4135: error is 0.6743103916447963
Epoch 4136: error is 1.6044336705521556
Epoch 4137: error is 0.6743573957467693
Epoch 4138: error is 0.6743506398671074
Epoch 4139: error is 0.6743439044705779
Epoch 4140: error is 0.67433718942957
Epoch 4141: error is 0.6743304946173152
Epoch 4142: error is 0.674323819907878
Epoch 4143: error is 0.6743171651761459
Epoch

In [51]:
print(r.weights)

[[ 2.52026436  0.05643019  2.51162473]
 [ 0.54960408 -8.09471632  3.98600548]
 [-0.76896759 -5.43912324  1.44739986]
 [ 4.92347391  1.73361288  3.91571134]
 [-0.59677216  7.75949122 -3.68036991]
 [ 0.35522313  3.27508886 -7.22594515]
 [-3.97144517 -2.46314976 -2.59697469]]


In [52]:
user = np.array([[0,0,0,1,1,0]])

In [53]:
print(r.run_visible(user))

[[1. 0.]]
